In [6]:
import pandas as pd
#from pandas.io.parsers import ParserError
import numpy as np
from helper import get_mapper
import json
import os
import re

In [7]:
from os import listdir, stat
from os.path import isfile, join
BASE_DIR = "."
MIN_SIZE = 512

In [8]:
def extract_blockidf(fullname):
    return fullname.split("Generation_DE ")[1].rsplit('[MW]')[0].strip()

def get_smard_name(f):
    return f.rsplit("/")[2].rsplit("_", 3)[0].strip()

In [9]:
def get_files_from_folder(folder):
    onlyfiles = [folder + "/" + f for f in listdir(folder) if isfile(join(folder, f))]
    onlyfiles.sort()
    files = [f for f in onlyfiles if stat(f).st_size > MIN_SIZE]
    return files

In [10]:
def convert2plantid(df, plantname):
    oldcols = list(df.columns)
    newcols = [extract_blockidf(x) for x in list(df.columns)[1:]]
    newcols2 = ['produced_at'] + [mapper[plantname][x] for x in newcols]
    test = dict(zip(oldcols, newcols2))
    result = df.rename(columns=test)
    #print(oldcols)
    #print(newcols2)
    return result

In [11]:
def get_plant_from_prod_name(prodname):
    tmp = ""
    try:
        tmp = mapper[prodname]
    except KeyError:
        print("plant " + prodname + " not found!")
        return None, None
    blocklist = tmp['list']
    block = blocklist[0]
    blockid = tmp[block]
    
    try:
        plantidx = bpm.loc[bpm.blockid == blockid, 'plantid'].item()
    except ValueError:
        return blockid, np.nan
    
    return blockid, plantidx

In [12]:
bpm = pd.read_csv("../basic/block_plant_mapper.csv")
bpm['plantid'] = bpm['plantid'].apply(lambda x: str(x).replace('/', '_'))

In [13]:
FOLDERS = [os.path.join(BASE_DIR, o) for o in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR,o))]
FOLDERS.sort()
FOLDERS = FOLDERS[1:-5]


In [14]:
FILES_L = [get_files_from_folder(f) for f in FOLDERS]
FILES = [item for sublist in FILES_L for item in sublist]

In [15]:
FOLDERS

['./2015', './2016', './2017', './2018', './2019', './2020']

In [16]:
mapper = get_mapper('../production/plantmapper.json')

In [22]:
#mapper

In [18]:
#bpm

In [19]:
#FILES

In [23]:
def get_year(fn):
    return int(fn.split("/")[1])

In [24]:
prod_mapper = []
for f in FILES:
    fn = get_smard_name(f)
    #print(fn)
    blockid, plantid = get_plant_from_prod_name(fn)
    year = 2015
    try:
        year = get_year(f)
    except IndexError:
        print(fn)
    prod_mapper.append([f, blockid, plantid, year])

plant Abwinden-Asten not found!
plant Altenw_rth not found!
plant Donaustadt not found!
plant Fernheizkraftwerk_Mellach not found!
plant Gerlos not found!
plant Greifenstein not found!
plant H_usling not found!
plant Kaprun_Hauptstufe not found!
plant Kaprun_Oberstufe not found!
plant Malta_Hauptstufe not found!
plant Malta_Oberstufe not found!
plant Mayrhofen not found!
plant Melk not found!
plant Ottensheim-Wilhering not found!
plant Riedersbach not found!
plant Ro_hag not found!
plant Schwarzach not found!
plant Timelkam_ not found!
plant Wallsee-Mitterkirchen not found!
plant Ybbs-Persenbeug not found!


In [26]:
get_plant_from_prod_name("Buschhaus")

('BNA0439', '03-01-01012110180')

In [27]:
get_year("./2015/Boxberg_201501010000_201512312345_71.csv")

2015

In [28]:
#prod_mapper 

In [29]:
prd_df = pd.DataFrame(prod_mapper, columns = ['file', 'blockid', 'plantid', 'year'])

In [30]:
#prd_df

In [ ]:
for index, row in list(prd_df.iterrows()):
    dfname = row[0]
    plantid = str(row[2])
    year = str(int(row[3]))
    smardname = get_smard_name(dfname)
    df = pd.read_csv(dfname, delimiter=";", parse_dates=[["Datum", "Uhrzeit"]])
    try:
        newdf = convert2plantid(df, smardname)
    except (IndexError, KeyError):
        print(smardname)
        continue
    #try:
    #print(dfname, year)
    #print(type(year))
    newdf.to_csv("./by_plantid/" + year + "/" + plantid + '.csv', index=False)
    #except TypeError:
    #    print(year, smardname)
    #print(newdf.dtypes)

Heizkraftwerk_Dresden-Nossener_Br_cke
Heizkraftwerk_Dresden-Nossener_Br_cke


In [ ]:
testdf = pd.read_csv("./2015/Boxberg_201501010000_201512312345_71.csv", delimiter=";", parse_dates=[["Datum", "Uhrzeit"]])

In [ ]:
testdf

In [ ]:
cols = list(testdf2.columns)[1:]

In [ ]:
convert2plantid(testdf, "Boxberg")

In [ ]:
testdf

In [ ]:
newcols2

In [ ]:
extract_blockidf('Generation_DE Q[MW]')